# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [27]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-cbba0a4300-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("World Wide Web") 

Consider the following exploratory information need:

> Investigate the origins of the World Wide Web and related academic activities and scientists.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q466`| World Wide Web      | node |





Also consider

```
wd:Q466 ?p ?obj .
```

is the BGP to retrieve all **properties of World Wide Web**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Find the inventors of World Wide Web (return IRI and name).

2. Identify the BGP for hypertext system.

3. Find all the hypertext systems born before the 1980 (return the IRI and name of the system and the inception date).

4. Identify the BGP for computer scientist

5. Find how many computer scientists there are for each continent (consider their citizenship). Return the IRI and name of the continent and the number of scientists for each continent. 

6. Find all the computer scientists who thaught at the University of Cambridge (return IRI and name)

7. Find all the computer scientists who wrote at least 5 books (please consider only the instances of book, exclude "literary work" or other type of work related to books). Return the IRI and name of the computer scientist and the number of books. 

## Task 1
Find the inventors of World Wide Web (return IRI and name).

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q466 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1430'), ('name', 'OpenPlaques subject ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('name', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2004'), ('name', 'NALT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 

Final query for this task

In [4]:
# use the specify IRI to find inventor
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q466 wdt:P61 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q80'), ('name', 'Tim Berners-Lee')]
[('obj', 'http://www.wikidata.org/entity/Q92749'), ('name', 'Robert Cailliau')]
2


## Task 2
Identify the BGP for hypertext system.

In [5]:
# write your queries
#find related object
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q466 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1668024'), ('name', 'service on internet')]
[('obj', 'http://www.wikidata.org/entity/Q1076968'), ('name', 'digital media')]
[('obj', 'http://www.wikidata.org/entity/Q35127'), ('name', 'website')]
[('obj', 'http://www.wikidata.org/entity/Q5460604'), ('name', 'Wikipedia:Vital articles')]
[('obj', 'http://www.wikidata.org/entity/Q75'), ('name', 'Internet')]
[('obj', 'http://www.wikidata.org/entity/Q80'), ('name', 'Tim Berners-Lee')]
[('obj', 'http://www.wikidata.org/entity/Q121182'), ('name', 'information system')]
[('obj', 'http://www.wikidata.org/entity/Q65966993'), ('name', 'hypertext system')]
[('obj', 'http://www.wikidata.org/entity/Q2385520'), ('name', 'ENQUIRE')]
[('obj', 'http://www.wikidata.org/entity/Q7742259'), ('name', 'The Interactive Encyclopedia System')]
[('obj', 'http://www.wikidata.org/entity/Q221989'), ('name', 'Deep web')]
[('obj', 'http://www.wikidata.org/entity/Q267091'), ('name', 'W3')]
[('obj', 'http://www.wikidata.

Final query for this task

In [6]:
# write your final query
#find specific goal
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q466 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER regex(?name , 'hypertext system').
}

LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q65966993'), ('name', 'hypertext system')]
1


## Task 3
Find all the hypertext systems born before the 1980 (return the IRI and name of the system and the inception date).

In [82]:
# write your querie
# with this querie we can find the IRI of inception and time. (P571 and P575)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    # bind something
   wd:Q466 ?p ?obj.
   # get the label
   ?p sc:name ?name.
   
   
}
LIMIT 35
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1430'), ('name', 'OpenPlaques subject ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('name', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2004'), ('name', 'NALT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 

wd:Q65966993

In [93]:
# write your queries
# get the useful propertise
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
    # bind something
   wd:Q466 ?p wd:Q65966993.
   # get the label
   ?p sc:name ?name.
   
   
}
LIMIT 35
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
1


Final query for this task

In [125]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name (YEAR(?when) as ?inception_date)
WHERE {
# bind something
    wd:Q65966993 wdt:P31 ?p.
    ?instance_of_hypertext wdt:P31 wd:Q65966993;
    wdt:P571 ?when.
    # get the label
    ?instance_of_hypertext sc:name ?name.
    FILTER(YEAR(?when) < 1980)
}
order by desc (?when)
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'Aspen Movie Map'), ('inception_date', '1978')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'Electronic Document System'), ('inception_date', '1978')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'ZOG'), ('inception_date', '1977')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'File Retrieval and Editing System'), ('inception_date', '1968')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'Hypertext Editing System'), ('inception_date', '1967')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'oNLine System'), ('inception_date', '1962')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'Project Xanadu'), ('inception_date', '1960')]
[('p', 'http://www.wikidata.org/entity/Q17155032'), ('name', 'Memex'), ('inception_date', '1939')]
8


## Task 4
Identify the BGP for computer scientist

In [8]:
# write your queries
# find sutible subject by using the filter
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p ?o.
   # get the label
   ?s sc:name ?name.
   FILTER REGEX(?name,'computer scientist').
}
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q82594'), ('name', 'computer scientist')]
1


Final query for this task

In [13]:
# write your final query
# find the related object 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q82594 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5157338'), ('name', 'computational scientist')]
[('obj', 'http://www.wikidata.org/entity/Q1650915'), ('name', 'researcher')]
[('obj', 'http://www.wikidata.org/entity/Q21198'), ('name', 'computer science')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q2476620'), ('name', 'list of computer scientists')]
[('obj', 'http://www.wikidata.org/entity/Q4991224'), ('name', 'Category:Computer scientists')]
[('obj', 'http://www.wikidata.org/entity/Q5157565'), ('name', 'computer professional')]
[('obj', 'http://www.wikidata.org/entity/Q166876'), ('name', 'Isidore of Seville')]
[('obj', 'http://www.wikidata.org/entity/Q66747123'), ('name', 'computer science terminology')]
[('obj', 'http://www.wikidata.org/entity/Q56604309'), ('name', 'IT professional')]
10


## Task 5
Find how many computer scientists there are for each continent (consider their citizenship). Return the IRI and name of the continent and the number of scientists for each continent.

In [15]:
# write your queries
# at first we find the continent IRI
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p ?obj .
   # get the label
   ?s sc:name ?name.
   FILTER REGEX(?name, 'continent')
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
[('s', 'http://www.wikidata.org/entity/Q8087154'), ('name', 'Category:10th century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8196566'), ('name', 'Category:2009 swine flu pandemic by continent')]
[('s', 'http://www.wikidata.org/entity/Q8208477'), ('name', 'Category:2nd century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8209910'), ('name', 'Category:3rd century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8210661'), ('name', 'Category:4th century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8211457'), ('name', 'Category:5th century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8212079'), ('name', 'Category:6th century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8213449'), ('name', 'Category:8th century BC by continent')]
[('s', 'http://www.wikidata.org/entity/Q8214335'), ('name', 'Category:9th century BC by continent')]

In [47]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q82594.
   # get the label
   ?p sc:name ?name.
   Filter regex(?p , 'field of work').
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
1


Final query for this task

In [28]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name ?nationality 
WHERE {
    # bind something
    ?p wdt:P101 wd:Q82594;
        wdt:P27 ?o.
    # get the label
    ?p  sc:name ?name.
    ?o sc:name ?nationality.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q7417239'), ('name', 'Sandy Douglas'), ('nationality', 'United Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q101044866'), ('name', 'Jordan J Bird'), ('nationality', 'United Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q92608'), ('name', 'Brian Kernighan'), ('nationality', 'Canada')]
[('p', 'http://www.wikidata.org/entity/Q5721658'), ('name', 'Henry Fuchs'), ('nationality', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q7563'), ('name', 'Joyce K. Reynolds'), ('nationality', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q92631'), ('name', 'Raymond Tomlinson'), ('nationality', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q14111'), ('name', 'Diane Pozefsky'), ('nationality', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q2387083'), ('name', 'Joel Spolsky'), ('nationality', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q16002

In [60]:
# write your final query
# by find the occupation we can reach to goal
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?s ?p wd:Q82594.
   # get the label
   ?p sc:name ?name.
   Filter regex(?name , 'occupation').
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
1


In [56]:
# write your queries
queryString = """
SELECT DISTINCT (count(?nationality) as ?num) ?continent ?cname
WHERE {
    # bind something
    ?person wdt:P101|wdt:P106 wd:Q82594;
    wdt:P27 ?nationality.
    ?nationality wdt:P30 ?continent.
    # get the label
    ?continent sc:name ?cname.
}
order by (?num)
LIMIT 15
"""
print("Results")
x=run_query(queryString)

Results
[('num', '23'), ('continent', 'http://www.wikidata.org/entity/Q5401'), ('cname', 'Eurasia')]
[('num', '76'), ('continent', 'http://www.wikidata.org/entity/Q15'), ('cname', 'Africa')]
[('num', '92'), ('continent', 'http://www.wikidata.org/entity/Q18'), ('cname', 'South America')]
[('num', '955'), ('continent', 'http://www.wikidata.org/entity/Q48'), ('cname', 'Asia')]
[('num', '2500'), ('continent', 'http://www.wikidata.org/entity/Q538'), ('cname', 'Insular Oceania')]
[('num', '2802'), ('continent', 'http://www.wikidata.org/entity/Q49'), ('cname', 'North America')]
[('num', '3496'), ('continent', 'http://www.wikidata.org/entity/Q46'), ('cname', 'Europe')]
7


## Task 6
Find all the computer scientists who thaught at the University of Cambridge (return IRI and name)

In [36]:
# write your queries
# at first we try to find Cambridge University
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    # bind something
    ?s ?p ?o.
    # get the label
    ?s sc:name ?name.
    FILTER Regex(?name, 'University of Cambridge')
}
LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8216424'), ('name', 'Category:A-Class University of Cambridge articles')]
[('s', 'http://www.wikidata.org/entity/Q8237338'), ('name', 'Category:Alumni of the University of Cambridge by college')]
[('s', 'http://www.wikidata.org/entity/Q8384467'), ('name', 'Category:Clubs and societies of the University of Cambridge')]
[('s', 'http://www.wikidata.org/entity/Q8703567'), ('name', 'Category:Organisations associated with the University of Cambridge')]
[('s', 'http://www.wikidata.org/entity/Q30679767'), ('name', 'Category:University of Cambridge Chancellor elections')]
[('s', 'http://www.wikidata.org/entity/Q35794'), ('name', 'University of Cambridge')]
[('s', 'http://www.wikidata.org/entity/Q41795465'), ('name', 'honorary doctor of the University of Cambridge')]
[('s', 'http://www.wikidata.org/entity/Q8882934'), ('name', 'Category:University of Cambridge examinations')]
[('s', 'http://www.wikidata.org/entity/Q8942672'), ('name', 'Category:Wiki

In [39]:
# Final query for this task
#from result of task we can use Q21198 for computer science
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
    # bind something
    ?s wdt:P101 wd:Q21198;
    wdt:P108 wd:Q35794.
    # get the label
    ?s sc:name ?sname.
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q92944'), ('sname', 'David Wheeler')]
[('s', 'http://www.wikidata.org/entity/Q7251'), ('sname', 'Alan Turing')]
[('s', 'http://www.wikidata.org/entity/Q46633'), ('sname', 'Charles Babbage')]
[('s', 'http://www.wikidata.org/entity/Q62857'), ('sname', 'Maurice Wilkes')]
[('s', 'http://www.wikidata.org/entity/Q92873'), ('sname', 'Roger Michael Needham')]
[('s', 'http://www.wikidata.org/entity/Q7821956'), ('sname', 'Tony Brooker')]
6


## Task 7
Find all the computer scientists who wrote at least 5 books (please consider only the instances of book, exclude "literary work" or other type of work related to books). Return the IRI and name of the computer scientist and the number of books.

In [99]:
# write your queries
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
    # bind something
    ?s ?p wd:Q82594.
    # get the label
    ?s sc:name ?name.
}
LIMIT 20
"""
print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q8190500'), ('name', 'Seinosuke Toda')]
[('s', 'http://www.wikidata.org/entity/Q603999'), ('name', 'Kevin Poulsen')]
[('s', 'http://www.wikidata.org/entity/Q11109023'), ('name', 'Michael Barr')]
[('s', 'http://www.wikidata.org/entity/Q20731777'), ('name', 'Valeria de Paiva')]
[('s', 'http://www.wikidata.org/entity/Q24901829'), ('name', 'Ernesto Damiani')]
[('s', 'http://www.wikidata.org/entity/Q61856956'), ('name', 'Richard Garner')]
[('s', 'http://www.wikidata.org/entity/Q6539960'), ('name', 'Liang-Jie Zhang')]
[('s', 'http://www.wikidata.org/entity/Q92669'), ('name', 'Jeffrey Shallit')]
[('s', 'http://www.wikidata.org/entity/Q15842362'), ('name', 'Richard Clippinger')]
[('s', 'http://www.wikidata.org/entity/Q15858435'), ('name', 'Hardo Bourguignon')]
[('s', 'http://www.wikidata.org/entity/Q15917430'), ('name', 'A. Ollongren')]
[('s', 'http://www.wikidata.org/entity/Q16018928'), ('name', 'Jim Thomas')]
[('s', 'http://www.wikidata.org/enti

Final query for this task

In [112]:
# write your final query
queryString = """
SELECT DISTINCT  ?p ?Bname ?Pname
WHERE {
    # bind something
    ?s wdt:P106|wdt:P101 wd:Q82594.
    # get the label
    ?p sc:name ?Bname.
    ?s sc:name ?Pname.
    
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P348'), ('Bname', 'software version identifier'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/P4669'), ('Bname', 'tabular software version'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/P747'), ('Bname', 'has edition or translation'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('Bname', 'Fonsi Nieto'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/Q100326534'), ('Bname', "Al Stewart's albums in chronological order"), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/Q100327667'), ('Bname', 'Category:Adelina Lopes Vieira'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/Q100343342'), ('Bname', 'Lubomír Stoklásek'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/Q1007576'), ('Bname', 'Ciudad Vieja'), ('Pname', 'Seinosuke Toda')]
[('p', 'http://www.wikidata.org/entity/Q100879156

In [168]:
queryString = """
SELECT DISTINCT ?subject ?p ?person_name 
WHERE {
    # bind something
    ?p wdt:P101|wdt:P106 wd:Q82594.
    ?book wdt:P50 ?p;
        wdt:P31 ?o.
    # get the label
    ?o sc:name ?subject.
    FILTER regex(?subject , 'scientific book').
}
order by desc(?booknumber)
limit 15
"""
print("Results")
x=run_query(queryString)

Results
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q20492609')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q46633')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q16517758')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q50177430')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q20492393')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q12253843')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q105467898')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q105161273')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q20492519')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q57083968')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/entity/Q107856074')]
[('subject', 'scientific book'), ('p', 'http://www.wikidata.org/enti

In [26]:
queryString = """
SELECT DISTINCT ?subject ?IRI ?name (count(?book)  as ?Num_of_Book )
WHERE {
    # bind something
    ?IRI wdt:P101|wdt:P106 wd:Q82594.
    ?book wdt:P50 ?IRI;
    wdt:P31 ?o.
    # get the label
    ?IRI sc:name ?name.
    ?o sc:name ?subject.
    FILTER regex(?subject , 'book').
    
    
}
order by desc(?Num_of_Book)
limit 30
"""
print("Results")
x=run_query(queryString)

Results
[('subject', 'scientific book'), ('IRI', 'http://www.wikidata.org/entity/Q20492519'), ('name', 'Olatz Arbelaitz'), ('Num_of_Book', '12')]
[('subject', 'scientific book'), ('IRI', 'http://www.wikidata.org/entity/Q16517758'), ('name', 'Iñaki Alegria'), ('Num_of_Book', '11')]
[('subject', 'scientific book'), ('IRI', 'http://www.wikidata.org/entity/Q50177430'), ('name', 'Kepa Sarasola Gabiola'), ('Num_of_Book', '7')]
[('subject', 'book review'), ('IRI', 'http://www.wikidata.org/entity/Q56860335'), ('name', 'Daniela Luzi'), ('Num_of_Book', '4')]
[('subject', 'book review'), ('IRI', 'http://www.wikidata.org/entity/Q58481435'), ('name', 'Alberto Salarelli'), ('Num_of_Book', '3')]
[('subject', 'scientific book'), ('IRI', 'http://www.wikidata.org/entity/Q12253843'), ('name', 'Arantza Diaz de Ilarraza Sanchez'), ('Num_of_Book', '3')]
[('subject', 'book review'), ('IRI', 'http://www.wikidata.org/entity/Q1293908'), ('name', 'Edward Tufte'), ('Num_of_Book', '2')]
[('subject', 'scientific bo